In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from datetime import timedelta

# --- CONFIGURATION & PARAMETERS ---
ASSETS = {
    'EURUSD': 'EURUSD=X', 'USDCHF': 'USDCHF=X', 'GBPUSD': 'GBPUSD=X',
    'USDCAD': 'USDCAD=X', 'BTCUSD': 'BTC-USD', 'ETHUSD': 'ETH-USD',
    'XAUUSD': 'XAUUSD=X', 'XAGUSD': 'XAGUSD=X', 'SP500m': '^GSPC', 'UK100': '^FTSE'
}

TIMEFRAME_SETTINGS = {
    '1WK': ('1wk', '10y'), 'D1': ('1d', '5y'), '4H': ('4h', '2y'),
    '1H': ('1h', '2y'), 'M30': ('30m', '60d'), 'M15': ('15m', '60d')
}

# Strategy Tuning
LOOKBACK_PERIOD = 14     # Period for average body calculation
BODY_MULTIPLIER = 1.5    # How much larger the candle must be than average
BOX_WIDTH = 30           # How many candles to trace the FVG forward
MOMENTUM_CHECK = 5       # Window to check for reversal candles
# Required candles in trend direction (e.g., 3 out of 5)
MOMENTUM_THRESHOLD = 1


def flatten_yf_columns(df):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    return df


def detect_fvg_logic(data):
    """
    Analyzes price data for FVGs and subsequent pullbacks/signals.
    """
    fvg_list = [None] * len(data)
    time_col = 'Datetime' if 'Datetime' in data.columns else 'Date'

    for i in range(2, len(data)):
        # 1. IDENTIFY FVG
        h1, l1 = data['High'].iloc[i-2], data['Low'].iloc[i-2]
        o2, c2 = data['Open'].iloc[i-1], data['Close'].iloc[i-1]
        h3, l3 = data['High'].iloc[i], data['Low'].iloc[i]

        mid_body = abs(c2 - o2)
        avg_body = abs(data['Close'] - data['Open']
                       ).iloc[max(0, i-1-LOOKBACK_PERIOD):i-1].mean()
        min_gap = mid_body * 0.10

        fvg_type, bottom, top = None, 0, 0

        # Bullish FVG
        if l3 > h1 and mid_body > (avg_body * BODY_MULTIPLIER) and (l3 - h1 > min_gap):
            fvg_type, bottom, top = 'bullish', h1, l3
        # Bearish FVG
        elif h3 < l1 and mid_body > (avg_body * BODY_MULTIPLIER) and (l1 - h3 > min_gap):
            fvg_type, bottom, top = 'bearish', l1, h3

        if fvg_type:
            signal_time, is_mitigated = None, False

            # 2. TRACE PULLBACKS WITHIN THE BOX_WIDTH
            check_range = min(i + BOX_WIDTH, len(data) - MOMENTUM_CHECK)
            for j in range(i + 1, check_range):
                curr_h, curr_l = data['High'].iloc[j], data['Low'].iloc[j]

                # Check for touch/retracement
                touched = (fvg_type == 'bullish' and curr_l <= top) or \
                          (fvg_type == 'bearish' and curr_h >= bottom)

                if touched:
                    is_mitigated = True

                    # Check for 3/5 Momentum Rejection
                    dir_count = 0
                    for k in range(j, j + MOMENTUM_CHECK):
                        op, cl = data['Open'].iloc[k], data['Close'].iloc[k]
                        if (fvg_type == 'bullish' and cl > op) or (fvg_type == 'bearish' and cl < op):
                            dir_count += 1

                    if dir_count >= MOMENTUM_THRESHOLD:
                        signal_time = data[time_col].iloc[j]
                        break

                    # Invalidation: If price closes through the orderflow zone
                    if (fvg_type == 'bullish' and data['Close'].iloc[j] < bottom) or \
                       (fvg_type == 'bearish' and data['Close'].iloc[j] > top):
                        break

            fvg_list[i] = {
                'type': fvg_type, 'bottom': bottom, 'top': top,
                'start': data[time_col].iloc[i-1], 'idx': i,
                'signal_time': signal_time, 'mitigated': is_mitigated
            }
    return fvg_list


def plot_fvg_signals(pair, timeframe, candle_limit=120):
    # Fetch Data
    interval, period = TIMEFRAME_SETTINGS[timeframe]
    df = yf.download(ASSETS[pair], period=period,
                     interval=interval, auto_adjust=False, progress=False)
    if df.empty:
        return

    df = flatten_yf_columns(df).reset_index()
    time_col = 'Datetime' if 'Datetime' in df.columns else 'Date'

    # Run Analysis
    df['FVG_RESULTS'] = detect_fvg_logic(df)
    df_plot = df.tail(candle_limit).copy()

    # Create Chart
    fig = go.Figure(data=[go.Candlestick(
        x=df_plot[time_col], open=df_plot["Open"], high=df_plot["High"],
        low=df_plot["Low"], close=df_plot["Close"], name=pair
    )])

    for _, row in df_plot.iterrows():
        res = row['FVG_RESULTS']
        if res:
            end_idx = min(res['idx'] + BOX_WIDTH, len(df) - 1)
            end_time = df[time_col].iloc[end_idx]

            # Neutralize color if mitigated, otherwise use trend colors
            if res['mitigated']:
                color = "rgba(180, 180, 180, 0.2)"
            else:
                color = "rgba(0, 255, 150, 0.3)" if res['type'] == 'bullish' else "rgba(255, 50, 50, 0.3)"

            fig.add_shape(type="rect", x0=res['start'], x1=end_time, y0=res['bottom'], y1=res['top'],
                          fillcolor=color, layer="below", line=dict(width=0))

            # Plot Signal Stars
            if res['signal_time'] and res['signal_time'] in df_plot[time_col].values:
                fig.add_trace(go.Scatter(
                    x=[res['signal_time']],
                    y=[res['top'] if res['type'] ==
                        'bullish' else res['bottom']],
                    mode="markers",
                    marker=dict(symbol="star", size=11, color="yellow",
                                line=dict(width=1, color="orange")),
                    name="Pullback Signal"
                ))

    fig.update_layout(
        title=f"<b>{pair} {timeframe}</b> - FVG Orderflow Analysis",
        template="plotly_dark",
        xaxis_rangeslider_visible=False,
        width=1200, height=750
    )
    fig.show()


if __name__ == "__main__":
    # Call with any pair and timeframe from the config
    # plot_fvg_signals('EURUSD', '1H', candle_limit=150)
    plot_fvg_signals('BTCUSD', '4H', candle_limit=100)

In [9]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import argrelextrema

# 1. Your Dictionaries
ASSETS = {
    'EURUSD': 'EURUSD=X', 'USDCHF': 'USDCHF=X', 'GBPUSD': 'GBPUSD=X',
    'USDCAD': 'USDCAD=X', 'BTCUSD': 'BTC-USD', 'ETHUSD': 'ETH-USD',
    'XAUUSD': 'XAUUSD=X', 'XAGUSD': 'XAGUSD=X', 'SP500m': '^GSPC', 'UK100': '^FTSE'
}

TIMEFRAME_SETTINGS = {
    '1WK': ('1wk', '10y'), 'D1': ('1d', '5y'), '4H': ('4h', '2y'),
    '1H': ('1h', '2y'), 'M30': ('30m', '60d'), 'M15': ('15m', '60d')
}


def analyze_divergence(asset_key, tf_key, candle_limit=100, order=5):
    """
    asset_key: e.g., 'BTCUSD'
    tf_key: e.g., '4H'
    candle_limit: number of candles to display
    """

    # Validation
    if asset_key not in ASSETS or tf_key not in TIMEFRAME_SETTINGS:
        print("Invalid Asset or Timeframe key.")
        return

    symbol = ASSETS[asset_key]
    interval, period = TIMEFRAME_SETTINGS[tf_key]

    # 1. Fetch Data
    print(f"Fetching {asset_key} on {tf_key} timeframe...")
    df = yf.download(symbol, period=period, interval=interval, progress=False)

    if df.empty:
        print("No data found.")
        return

    # Flatten columns if multi-index
    df.columns = [col[0] if isinstance(
        col, tuple) else col for col in df.columns]

    # 2. Indicators
    df['RSI'] = ta.rsi(df['Close'], length=14)
    df.dropna(inplace=True)

    # 3. Detect Signals
    # We calculate on the whole set, but identify signals
    # within the last 'candle_limit' to keep it fast and relevant.
    df['Bullish_Div'] = False
    df['Bearish_Div'] = False

    max_idx = argrelextrema(df['Close'].values, np.greater, order=order)[0]
    min_idx = argrelextrema(df['Close'].values, np.less, order=order)[0]

    # Bearish Divergence Logic
    for i in range(1, len(max_idx)):
        curr, prev = max_idx[i], max_idx[i-1]
        if df['Close'].iloc[curr] > df['Close'].iloc[prev] and df['RSI'].iloc[curr] < df['RSI'].iloc[prev]:
            df.at[df.index[curr], 'Bearish_Div'] = True

    # Bullish Divergence Logic
    for i in range(1, len(min_idx)):
        curr, prev = min_idx[i], min_idx[i-1]
        if df['Close'].iloc[curr] < df['Close'].iloc[prev] and df['RSI'].iloc[curr] > df['RSI'].iloc[prev]:
            df.at[df.index[curr], 'Bullish_Div'] = True

    # 4. Limit the display
    df_plot = df.tail(candle_limit)

    # 5. Plotting
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.03, row_heights=[0.7, 0.3])

    # Price Chart
    fig.add_trace(go.Candlestick(x=df_plot.index, open=df_plot['Open'], high=df_plot['High'],
                                 low=df_plot['Low'], close=df_plot['Close'], name='Price'), row=1, col=1)

    # Signal markers
    bull = df_plot[df_plot['Bullish_Div']]
    bear = df_plot[df_plot['Bearish_Div']]

    fig.add_trace(go.Scatter(x=bull.index, y=bull['Low'] * 0.995, mode='markers',
                             marker=dict(symbol='triangle-up', size=12, color='#00ff00'), name='Bull Div'), row=1, col=1)
    fig.add_trace(go.Scatter(x=bear.index, y=bear['High'] * 1.005, mode='markers',
                             marker=dict(symbol='triangle-down', size=12, color='#ff0000'), name='Bear Div'), row=1, col=1)

    # RSI Chart
    fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot['RSI'], line=dict(
        color='#ff9900', width=2), name='RSI'), row=2, col=1)
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

    fig.update_layout(title=f"{asset_key} ({tf_key}) - Last {candle_limit} Candles",
                      template='plotly_dark', xaxis_rangeslider_visible=False, height=800)
    fig.show()


# --- CALL AS REQUESTED ---
# analyze_divergence('BTCUSD', '1H', candle_limit=100)
analyze_divergence('EURUSD', 'M30', candle_limit=100)

Fetching EURUSD on M30 timeframe...


/var/folders/pp/x__1tf4n7j98nw_8bwlmgxj00000gn/T/ipykernel_10575/1741925546.py:39: FutureWarning:

YF.download() has changed argument auto_adjust default to True

